In [10]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fccf834dc08ca980a43ae9f999888c77ccc33ad2d4e167c9dbc0850186c690b6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [12]:
import csv
import pymorphy2

# Инициализация pymorphy2 для морфологического анализа
morph = pymorphy2.MorphAnalyzer()


class SearchEngineCSV:
    def __init__(self, csv_file):
        # Чтение и загрузка данных из CSV
        self.sentences = []
        self.tokens = []
        self.lemmas = []
        self.pos_tags = []
        self.chapters = []  # Новое поле для хранения источников

        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Пропуск заголовка
            for row in reader:
                self.sentences.append(row[0])  # Предложения
                self.tokens.append(row[1].split(';'))  # Токены
                self.lemmas.append(row[2].split(';'))  # Леммы
                self.pos_tags.append(row[3].split(';'))  # Части речи
                self.chapters.append(row[4])  # Источник (Chapter)

    def match_token(self, token, word, lemma, pos):
        """Проверка совпадения токена с запросом."""
        if '"' in token:
            return token.strip('"') == word

        if '+' in token:
            query_word, query_pos = token.split('+')
            return lemma == query_word and pos == query_pos

        if token.isupper():
            return pos == token

        return lemma == morph.parse(token)[0].normal_form

    def match_sequence(self, query_tokens, words, lemmas, pos_tags):
        """Проверка совпадения последовательности токенов."""
        for j, query_token in enumerate(query_tokens):
            if query_token.isupper():
                # Если запрос — это POS-тег, проверяем только POS-тег
                if pos_tags[j] != query_token:
                    return False
            else:
                # Иначе обрабатываем стандартные правила совпадения
                if not self.match_token(query_token, words[j], lemmas[j], pos_tags[j]):
                    return False
        return True

    def search(self, query):
        query_tokens = query.split()

        matches = []
        for idx, sentence in enumerate(self.sentences):
            words = self.tokens[idx]
            lemmas = self.lemmas[idx]
            pos_tags = self.pos_tags[idx]

            # Поиск последовательности токенов в предложении
            for i in range(len(words) - len(query_tokens) + 1):
                if len(words) != len(lemmas) or len(words) != len(pos_tags):
                    continue
                # Проверка последовательности для n-граммы
                if self.match_sequence(query_tokens, words[i:i+len(query_tokens)], lemmas[i:i+len(query_tokens)], pos_tags[i:i+len(query_tokens)]):
                    # Форматируем предложение с источником
                    source_info = f"[{self.chapters[idx]}]"
                    formatted_sentence = f"{sentence} {source_info}"
                    matches.append(formatted_sentence)
                    break

        return matches

# Пример использования
csv_file = '/content/corpus_data_with_labels.csv'  # Путь к файлу
search_engine = SearchEngineCSV(csv_file)

# Примеры запросов
print(search_engine.search('сказать'))  # Любая форма слова "сказать"
print(search_engine.search('"меня"'))  # Точная форма "меня"
print(search_engine.search('человек+NOUN'))  # Лемма "человек" с POS-тегом "S"
print(search_engine.search('NOUN ADV VERB'))  # Существительное, глагол, наречие в последовательности

['Вы мне опять скажете, что человек не может быть так дурен, а я вам скажу, что ежели вы верили возможности существования всех трагических и романтических злодеев, отчего же вы не веруете в действительность Печорина? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', 'Вы скажете, что нравственность от этого не выигрывает? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', '— Скажите, пожалуйста, отчего это вашу тяжелую тележку четыре быка тащат шутя, а мою пустую шесть скотов едва подвигают с помощию этих осетин? [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', 'Я сказал ему. [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', '— сказал он: — и хлеба по-русски назвать не умеет, а выучил: «офицер, дай на водку!». [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermont